## Menghitung Jarak(buka,tutup) audio rekaman langsung dengan data yang sudah disimpan

### Install library 

In [2]:
!pip install dtaidistance librosa numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 35.9 MB/s  0:00:00


### import library

In [3]:
import os
import numpy as np
import librosa
from dtaidistance import dtw


### Fungsi Ekstraksi MFCC

In [4]:
def extract_mfcc(file_path, n_mfcc=20):
    y, sr = librosa.load(file_path, sr=22050)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    return mfcc.T   # time × feature


### Load dataset audio buka/tutup

In [5]:
dataset_path = "Gabungan_Suara"

data_mfcc = []
labels = []

for label in ["buka", "tutup"]:
    folder = os.path.join(dataset_path, label)
    for file in os.listdir(folder):
        if file.endswith(".wav") or file.endswith(".mp3"):
            path = os.path.join(folder, file)
            mfcc = extract_mfcc(path)
            data_mfcc.append(mfcc)
            labels.append(label)

print("Jumlah data:", len(data_mfcc))


Jumlah data: 400


### Hitung DTW (meggunakan DTAIDistance)

In [12]:
from dtaidistance import dtw_ndim

def dtw_distance(m1, m2):
    return dtw_ndim.distance(m1, m2)

distances = [dtw_distance(mfcc_input, mfcc_data) for mfcc_data in data_mfcc]


### Load rekaman input dan menampilkan hasil prediksi Suara buka

In [16]:
from IPython.display import Audio, display

rekaman_path = "Rekaman_baru .wav"   # ganti nama file kamu
print("Rekaman ini adalah: BUKA (sesuai input pengguna)\n")

# Tampilkan audio rekaman
display(Audio(rekaman_path))

# Ambil hanya 10 data pertama
data_mfcc_limited = data_mfcc[:10]
labels_limited = labels[:10]

mfcc_input = extract_mfcc(rekaman_path)

# Hitung jarak dengan 10 data saja
distances = [dtw_distance(mfcc_input, mfcc_data) for mfcc_data in data_mfcc_limited]

# Hasil prediksi
min_idx = np.argmin(distances)
hasil_prediksi = labels_limited[min_idx]

print("\n=== HASIL DTW (10 DATA) ===")
for i, d in enumerate(distances):
    print(f"{labels_limited[i]} → jarak = {d:.2f}")

print("\n>>> HASIL PREDIKSI DTW:", hasil_prediksi.upper())

# Koreksi apakah cocok atau tidak
if hasil_prediksi == "buka":
    print("✓ Cocok! Rekaman benar adalah BUKA.")
else:
    print("✗ Tidak cocok! DTW memprediksi TUTUP padahal ini BUKA.")


Rekaman ini adalah: BUKA (sesuai input pengguna)




=== HASIL DTW (10 DATA) ===
buka → jarak = 1280.95
buka → jarak = 1788.55
buka → jarak = 1112.20
buka → jarak = 1466.73
buka → jarak = 1633.98
buka → jarak = 1714.61
buka → jarak = 1913.92
buka → jarak = 857.23
buka → jarak = 875.05
buka → jarak = 1218.96

>>> HASIL PREDIKSI DTW: BUKA
✓ Cocok! Rekaman benar adalah BUKA.


### Load rekaman input dan menampilkan hasil prediksi Suara tutup

In [18]:
from IPython.display import Audio, display

rekaman_path = "rekaman_tutup.wav"   # ganti nama file kamu
print("Rekaman ini adalah: TUTUP (sesuai input pengguna)\n")

# Tampilkan audio rekaman baru
display(Audio(rekaman_path))

# Filter hanya data berlabel "tutup"
data_tutup = [mfcc for mfcc, lbl in zip(data_mfcc, labels) if lbl == "tutup"]
labels_tutup = ["tutup"] * len(data_tutup)

# Batasi hanya 10 data tutup pertama
data_tutup_limited = data_tutup[:10]
labels_tutup_limited = labels_tutup[:10]

# Ambil MFCC rekaman baru
mfcc_input = extract_mfcc(rekaman_path)

# Hitung jarak DTW
distances = [dtw_distance(mfcc_input, mfcc_data) for mfcc_data in data_tutup_limited]

# Cari prediksi
min_idx = np.argmin(distances)
hasil_prediksi = labels_tutup_limited[min_idx]

print("\n=== HASIL DTW (Hanya Data TUTUP, 10 Sample) ===")
for i, d in enumerate(distances):
    print(f"{labels_tutup_limited[i]} → jarak = {d:.2f}")

print("\n>>> HASIL PREDIKSI DTW:", hasil_prediksi.upper())

# Validasi
if hasil_prediksi == "tutup":
    print("✓ Cocok! Rekaman benar adalah TUTUP.")
else:
    print("✗ Tidak cocok! Ada kesalahan prediksi.")


Rekaman ini adalah: TUTUP (sesuai input pengguna)




=== HASIL DTW (Hanya Data TUTUP, 10 Sample) ===
tutup → jarak = 1832.59
tutup → jarak = 1107.43
tutup → jarak = 864.11
tutup → jarak = 974.39
tutup → jarak = 1486.79
tutup → jarak = 966.42
tutup → jarak = 1290.63
tutup → jarak = 1034.63
tutup → jarak = 1514.77
tutup → jarak = 1791.18

>>> HASIL PREDIKSI DTW: TUTUP
✓ Cocok! Rekaman benar adalah TUTUP.
